# Requirment 

In [ ]:
%python.exe -m pip install --upgrade pip

In [216]:
%pip install pytesseract
%pip install beautifulsoup4
%pip install pandas
# %pip install tensorflow
%pip install Pillow
%pip install lxml
%pip install opencv-python
%pip install pathlib
%pip install ast

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\User\AppData\Local\Temp\pip-install-cmvjdell\ast_af27a5a016f54dbf817e7d342a0d55fb\setup.py", line 6, in <module>
          README = codecs.open(os.path.join(here, 'AST/README'), encoding='utf8').read()
        File "C:\Users\User\AppData\Local\Programs\Python\Python38\lib\codecs.py", line 905, in open
          file = builtins.open(filename, mode, buffering)
      FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-cmvjdell\\ast_af27a5a016f54dbf817e7d342a0d55fb\\AST/README'
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encount

# Import

In [119]:
from bs4 import BeautifulSoup
import base64
import pandas as pd
# import tensorflow as tf
from pytesseract import pytesseract
from PIL import Image
import glob
import cv2
import numpy as np
from pathlib import Path
import os
import json
import collections

# Main

In [225]:
%cd C:\Users\User\Desktop\backup\brt\Projects\Crawing-List-of-Admitted-Schools-for-Students\
    
path_to_tesseract = Path(r'C:\Program Files\Tesseract-OCR\tesseract.exe')
pytesseract.tesseract_cmd = path_to_tesseract


for file in glob.iglob('data\department\*交叉查榜*.html'):
    # print(file)
    processing_department(file)


c:\Users\User\Desktop\backup\brt\Projects\Crawing-List-of-Admitted-Schools-for-Students
長庚大學中醫學系
list_stuno_depts: ['私立復旦高中學測查榜- 112年大學 _ 科大交叉查榜-0www.com.tw.html', [1, 5, ['中國醫藥大學 中醫學系甲組', '中國醫藥大學 中醫學系乙組', '國立陽明交通大學 醫學系(公費生)', '長庚大學 中醫學系'], 'iVBORw0KGgoAAAANSUhEUgAAAFoAAAAUCAIAAACCmL1JAAAABnRSTlMA/wD/AP83WBt9AAAACXBIWXMAAA7EAAAOxAGVKw4bAAAA9klEQVRYhe2W3Q7DIAiFYdn7vzK7MCXlR0Tn2ibju2uPKJwqFokIioPX3Qk8i7JDUHYIyg5B2SF49wREBAD33pmV2ksmUFlSITZwLWqIb4c775qkDEJERFQFtEclQb+StagMzmHZ6AUAEJHNtQ1WTiXLcKOCxKZwdkewwJp0Jjho7nuecMo1G5XkulbadjV0amZVvefN1TPazpmJ6iaZOZ9bJDtAfcM4NmjSyeUyXH3R2o+23AVmS83w0P8O9+yoAeBtli976s/tUCmey1DbISnZRyaOSmUbNHO1zC4p8xsWSMGNPpxwSLeV/icP7R13UXYIyg7BByMX1UBx0uWlAAAAAElFTkSuQmCC'], [1, 5, ['國立成功大學 醫學系', '高雄醫學大學 醫學系', '中國醫藥大學 醫學系', '長庚大學 醫學系', '臺北醫學大學 醫學系', '國立臺灣科技大學化學工程系'], 'iVBORw0KGgoAAAANSUhEUgAAAFoAAAAUCAIAAACCmL1JAAAABnRSTlMA/wD/AP83WBt9AAAACXBIWXMAAA7EAAAOxAGVKw4bAAAA9ElEQVRYhe2W2w7EIAhEYdP//2X2wUgUcWrVbZss503oKEzqhUWEgszn6QLeRdhREXZUhB0VYUfF0UswMxG5987VVAoqIKspIwHCMji

KeyboardInterrupt: 

## Functions

In [217]:
def processing_department(filename):
    with open(filename, 'r', encoding="utf-8") as file:
        html = file.read()
        # print(html)
        soup = BeautifulSoup(html, 'html.parser')
    
    fields = ['rank', 'exam_location', 'stuno', 'univ',  'year', 'department_name', 'select']
    df = pd.DataFrame(columns=fields)
    df1 = pd.DataFrame(columns=fields)
    df2 = pd.DataFrame(columns=fields)
    
    department_name = []
    select = []

    str = soup.head.find_all('title')[0].text
    pos = str.find('年')
    year = str[pos-3:pos]  # str[5:8] str[5], str[6], str[7]

    str = soup.head.find_all('title')[0].text
    pos = str.find('-')
    univ = str[:pos].rstrip()
    

    dark_list = soup.body.find_all(bgcolor="#DEDEDC") # 褐色
    list_of_color = 'dark_list'
    df1 = processing_list(univ, year, dark_list, df, department_name, select, list_of_color)
    
    white_list = soup.body.find_all(bgcolor="#FFFFFF") # 白色
    list_of_color = 'white_list'
    df2 = processing_list(univ, year, white_list, df, department_name, select, list_of_color)
    
    # make two df cross-interleave
    df = cross_interleave(df1, df2)

    # explode list into df
    df = df.explode(['department_name', 'select']).reset_index(drop=True)
    # df = df.dropna()
    
    # save to csv
    result_dir = Path(r'.\result')
    df.to_csv(path_or_buf = result_dir / f'{univ}.csv', encoding='utf-8-sig', index=False)
    

### Functions

In [216]:
def processing_list(univ, year, color_list, df, department_name, select, list_of_color):
    rank = ''
    stuno = ''
    print(univ)
    for item in color_list:
        color_list_item_scope1 = item.children # get message under the soup.body.find_all(bgcolor="")
        count_item = 0
        exam_location = ""
        for item_scope2 in color_list_item_scope1:
            # print("i:", item_scope2," ", count_item)
            """
            count_item:3 = 正備取
            count_item:5 = number
            count_item:9 = 錄取校系
            """
            if count_item == 3:
                rank = img_detect(item_scope2 = item_scope2, img = r'data/rank.png', langage='chi_sim', oem_psm = '--oem 0 --psm 7')

            elif count_item == 5:
                stuno_third_ori = ''
                stuno_last_ori = ''

                try:
                    base64_str = item_scope2.find_all('img')[0]['src'].split(',')[1]
                except IndexError:
                    return "None"
                else:
                    with open(r'data/num.png', "wb") as fh:
                        fh.write(base64.decodebytes(bytes(base64_str, 'utf-8')))
                    img_ = cv2.imread(r'data/num.png')
                    for i in range(3,4): # to get what digit you want
                        stuno_third_ori = int(img_detect_crop(img_, i))
                    for i in range(8,9): # to get what digit you want
                        stuno_last_ori = int(img_detect_crop(img_, i))

                # get exam location
                x = item.find_all('a')
                exam_location = x[0].text[6:-1]

            elif count_item == 9:
                tr_scope3 = item_scope2.find_all('tr')
                department_name = []
                select = []
                for item_scope4 in tr_scope3:
                    # print("item_scope4:", count, rank, item_scope4)
                    count_item_scope5 = 0
                    for item_scope5 in item_scope4:
                        # print('item_scope5:', count_, item_scope5)
                        """_summary_
                        count_item_scope5: 1 徽章
                        count_item_scope5: 3 大學
                        count_item_scope5: 5 校系排名
                        """
                        if count_item_scope5 == 1:
                            img_tag = item_scope5.find('img')
                            # print(img_tag)
                            if img_tag is not None:
                                img_tag_ = True
                            else:
                                img_tag_ = False
                            select.append(img_tag_)
                            
                        elif count_item_scope5 == 3:
                            # We'll drop later to suit up with img_tag_
                            if item_scope5.text != "":
                                department_name.append(item_scope5.text)
                            else:
                                select.pop()
                                # department_name.append(np.nan)
                            
                        elif count_item_scope5 == 5:
                            # to get the rank of each dept
                            pass
                        
                        count_item_scope5 += 1
                ## TO get all data
                """
                count_item:3 = 正備取
                count_item:5 = number
                count_item:9 = 錄取校系: 徽章、大學、校系排名
                """
                # department_name
                
                # print('department_name:', department_name)
                # print(f'stuno_third_ori: {stuno_third_ori} stuno_last_ori: {stuno_last_ori}')
            
                stuno = get_whole_number(list_of_color, exam_location, stuno_third_ori, stuno_last_ori, department_name)
                print('get_whole_number:', stuno)
                
            count_item += 1
        
        dict = {'rank': rank, 'exam_location': exam_location, 'stuno': stuno, 'univ': univ, 'year': year, 'department_name': department_name, 'select': select}
        df = pd.concat([df, pd.DataFrame([dict])], ignore_index = True)
    
    return df

def cross_interleave(df1, df2):
    # make two df cross-interleave
    df = pd.DataFrame()
    max_len = max(len(df1), len(df2))
    for line in range(max_len):
        if line < len(df1):
            df = pd.concat([df, df1.iloc[[line]]], ignore_index=True)
        if line < len(df2):
            df = pd.concat([df, df2.iloc[[line]]], ignore_index=True)
    return df

#### Function

In [226]:
def img_detect(item_scope2, img, langage, oem_psm):
    try:
        base64_str = item_scope2.find_all('img')[0]['src'].split(',')[1]
    except IndexError:
        return "None"
    else:
        with open(img, "wb") as fh:
            fh.write(base64.decodebytes(bytes(base64_str, 'utf-8')))
        path_to_image = img
        cv2.imwrite(img, thresholding_(path_to_image))
        return pytesseract.image_to_string(Image.open(path_to_image), lang=langage, config=oem_psm)
    
def img_detect_crop(img_, i):
    # start from (4, 20)
    basic_width = 4
    # 9 pixels each
    digit_width = 9
    
    crop_initial = digit_width*(i-1)+basic_width
    crop_end = digit_width*i+basic_width
    crop_img = img_[0: 20, crop_initial:crop_end]
    cv2.imwrite(f'data/crop_img{i}.png', crop_img)
    image = f'data/crop_img{i}.png'
    path_to_image = image
    cv2.imwrite(f'data/crop_img{i}.png', thresholding_(image))
    return pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')

def get_whole_number(list_of_color, exam_location, stuno_third_ori, stuno_last_ori, department_name):
    for file in glob.iglob(f'data\\test_location\\*{exam_location}學測查榜*'):
        exam_location_file_name = os.path.basename(file)
        
        list_stuno_depts = whole_num_processing(list_of_color, file)
        list_stuno_depts.insert(0, exam_location_file_name)
        print('list_stuno_depts:', list_stuno_depts)
        

        # list_stuno_depts is ['filename',[num_third, num_last, [dept1, dept2, dept3]], [num_third, num_last, [dept1, dept2, dept3]]]
        # 這邊有讀校系迴圈
        # for j in range(3):
        for i in range(1, len(list_stuno_depts)): # aware of list[x>1]
            # print(list_stuno_depts[i])
            if stuno_third_ori == list_stuno_depts[i][0] and stuno_last_ori == list_stuno_depts[i][1] \
                and collections.Counter(department_name) == collections.Counter(list_stuno_depts[i][2]):
                    exam_loca_num = ''
                    with open(r'data/exam_loca_exact_num.png', "wb") as fh:
                        fh.write(base64.decodebytes(bytes( list_stuno_depts[i][3], 'utf-8')))
                    img__ = cv2.imread(r'data/exam_loca_exact_num.png')
                    for j in range(1,9):
                        exam_loca_num = str(exam_loca_num + img_detect_crop(img__, j)).strip()
                    # print(exam_loca_num)
                    return exam_loca_num
                        
                    
    # get exam_lacation stuno & list of univs, merge it into a list -> [stuno[0:3], stuno[-1], dept....]
    # make the same of the original version
    # if list_exam_location[i] == list_original[i]:
    # get the whole number and return

##### Function

In [214]:
def thresholding_(png):
    image = cv2.imread(png)
    gray = get_grayscale(image)
    thresh_ = thresholding(gray)
    return thresh_
  
def whole_num_processing(list_of_color, file):
    with open(file, 'r', encoding="utf-8") as file:
        html = file.read()
        # print(html)
        soup = BeautifulSoup(html, 'html.parser')

    whole_list = []
    
    dark_list = soup.body.find_all(bgcolor="#DEDEDC") # 褐色
    list_add_dark = find_number(dark_list)

    white_list = soup.body.find_all(bgcolor="#FFFFFF") # 白色
    list_add_white = find_number(white_list)
    
    if list_add_dark:
        if list_add_white:
            whole_list = list_add_dark + list_add_white
        else:
            whole_list = list_add_dark
    else:
        whole_list = list_add_white

    return whole_list

###### Function

In [213]:
# Convert the image to grayscale
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Remove noise from the image
def remove_noise(image):
    return cv2.medianBlur(image, 5)
 
# Thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

# Dilation
def dilate(image):
    kernel = np.ones((5, 5), np.uint8)
    return cv2.dilate(image, kernel, iterations=1)
    
# Erosion
def erode(image):
    kernel = np.ones((5, 5), np.uint8)
    return cv2.erode(image, kernel, iterations=1)

# Opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5, 5), np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

# Canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

# Skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]

    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

# Template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)

#### find number

In [212]:
def find_number(color_list):
    all_list = []
    for item in color_list:
        color_list_item_scope1 = item.children # get message under the soup.body.find_all(bgcolor="")
        count_item = 0
        list_dept = []

        for item_scope2 in color_list_item_scope1:
            """
            count_item3: number
            count_item7: 校系
            """
            if count_item == 3:        
                stuno_third = ''
                stuno_last = ''            
                try:
                    base64_str = item.find_all('img')[0]['src'].split(',')[1]
                except IndexError:
                    return "None"
                else:
                    with open(r'data/exam_loca_num.png', "wb") as fh:
                        fh.write(base64.decodebytes(bytes(base64_str, 'utf-8')))
                    img__ = cv2.imread(r'data/exam_loca_num.png')
                    for i in range(3,4):
                        stuno_third = img_detect_crop(img__, i)
                    for i in range(8,9):
                        stuno_last = img_detect_crop(img__, i)

            elif count_item == 7: # 校系
                x = item.find_all('a')
                for i in x:

                    # if i.text != "":
                    list_dept.append(i.text)
                    # else: 
                    #     list_dept.append(np.nan)
            count_item += 1
        
        # 先把長庚大學的拉出來 -> 算數字 -> 比對數字 -> 比對學校
        is_CGU_exist = False
        for list_item in list_dept:
            if list_item[0:4] == "長庚大學":
                is_CGU_exist = True
            else: pass

        if is_CGU_exist:
            all_list.append([int(stuno_third), int(stuno_last), list_dept, base64_str])
            
        # 先存img碼等最後再分析圖片省時間
        # not return here but create a variable to save all the lists
        
    return all_list

# Number

### data_format

In [65]:
str1 = 'lL1x%\"«oO'

char_to_replace = {'l':'1', 'L':'1', 'x':'*', '%':'*', '\"':'*', '«':'*', 'o':'0', 'O':'0'}

# Iterate over all key-value pairs in dictionary 
for key, value in char_to_replace.items():
    # Replace key character with value character in string
    str1 = str1.replace(key, value)     

print(str1)
print(type(str1))

111****00
<class 'str'>


### test img_cropping

In [ ]:
from pytesseract import pytesseract
from PIL import Image
import cv2
from pathlib import Path

# target
img = cv2.imread(r'C:\Users\User\Desktop\backup\brt\Projects\Crawing-List-of-Admitted-Schools-for-Students\data\num.png')

print(img.shape)

# 從(4, 20)開始
basic_width = 4
# 每個間格9 pixel
digit_width = 9

rlt = ''

def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]


# 8 digits
for i in range(1, 9):
    crop_initial = digit_width*(i-1)+basic_width
    crop_end = digit_width*i+basic_width
    crop_img = img[0: 20, crop_initial:crop_end]
    print(crop_img.shape)
    print(type(i), i)
    cv2.imwrite(f'data/crop_img{i}.png', crop_img)
    # cv2.imshow("img", crop_img)
    # cv2.waitKey(0)
    image = f'data/crop_img{i}.png'
    path_to_image = image
    image_ = cv2.imread(image)
    gray = get_grayscale(image_)
    thresh_ = thresholding(gray)
    cv2.imwrite(f'data/crop_img{i}.png', thresh_)
    path_to_tesseract = Path(r'C:\Program Files\Tesseract-OCR\tesseract.exe')
    pytesseract.tesseract_cmd = path_to_tesseract    
    rlt = str(rlt + pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')).strip()

print(rlt)
